One problem with the automatic fiducial detection is that it is very slow for large fields of view (it can take hours). The slow down comes from the linking across frames. This is problematic because one may need to try different parameters to achieve good fiducial detection.

A possible solution is to visually select regions containing fiducials from histogram plots. Then, the linking can be made to occur only over the search region.

In [1]:
%pylab
import DataSTORM.processors as ds
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


### Clean up the data

In [2]:
filename = Path('../test-data/MicroTubules_LargeFOV/FOV1_1500_10ms_1_MMStack_locResults.dat')
with open(str(filename.resolve()), 'r') as file:
    df = pd.read_csv(file, engine = 'c')

/home/douglass/anaconda3/envs/DataSTORM/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
stringMask             = df['uncertainty [nm]'].map(lambda x: isinstance(x, str)).as_matrix()
df['uncertainty [nm]'] = pd.to_numeric(df['uncertainty [nm]'], errors='coerce')
df.replace([np.inf, -np.inf], np.nan, inplace = True)
df.dropna()
df.reindex()
print('Done.')

Done.


# Display 2D histograms
2D histograms seem to be a good way to identify fiducials, especially in microtubules and the centriole samples. This is because the fiducials don't drift much and should be present in every frame, leading to large values in the bins that hide the other features.

Let's first try to generate a nice looking 2D histogram that tells us the information we need, specifically the location of the fiducials.

In [37]:
toMicrons = .001 # Conversion of columns to microns

fig, ax = plt.subplots()
im      = ax.hexbin(df['x [nm]'] * toMicrons, df['y [nm]'] * toMicrons, gridsize = 100, cmap = plt.cm.YlOrRd_r)
ax.set_xlabel(r'x-position, $\mu m$')
ax.set_ylabel(r'y-position, $\mu m$')
ax.invert_yaxis()

cb      = plt.colorbar(im)
cb.set_label('Counts')

plt.show()

In [33]:
plt.close()

In [34]:
df['x [nm]'] * 0.001

0           0.15105
1           0.36718
2           0.42242
3           0.51929
4           0.59009
5           0.68517
6           0.60730
7           0.70178
8           0.82221
9           0.74504
10          0.88776
11          0.90477
12          0.90338
13          0.96235
14          0.99653
15          0.91937
16          1.06840
17          1.04870
18          1.23830
19          1.11920
20          1.13020
21          1.41070
22          1.39880
23          1.46780
24          1.47730
25          1.70830
26          1.71900
27          1.77840
28          1.66790
29          1.69460
             ...   
8688753    55.72000
8688754    55.95400
8688755    56.75800
8688756    56.82200
8688757    57.80700
8688758    58.16000
8688759    58.37400
8688760    58.67500
8688761    58.86100
8688762    59.47900
8688763    59.66400
8688764    60.73400
8688765    61.27500
8688766    61.39600
8688767    61.50000
8688768    61.45900
8688769    61.80600
8688770    61.75200
8688771    62.40000
